# Before you start :
   - These exercises are related to the Pivot table and correlation lessons.
   - Keep in mind that you need to use some of the functions you learned in the previous lessons.
   - All datasets are provided in the `your-code` folder of this lab.
   - Elaborate your codes and outputs as much as you can.
   - Try your best to answer the questions and complete the tasks and most importantly enjoy the process!!!

#### Import all the libraries that are necessary.

In [69]:
# your answer here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression

# Challenge 1

#### Open the ` Employee.csv` file and name your dataset `employee`.

In [70]:
# your answer here

employee=pd.read_csv(r"data\Employee.csv")
employee.head(5)

,Name,Department,Education,Gender,Title,Years,Salary
0,Jose,IT,Bachelor,M,analyst,1,35
1,Maria,IT,Master,F,analyst,2,30
2,David,HR,Master,M,analyst,2,30
3,Sonia,HR,Bachelor,F,analyst,4,35
4,Samuel,Sales,Master,M,associate,3,55


#### What's the mean salary by department?
Hint : There are 2 ways to do it - one way is with groupby and the other way is with pivot tables. Do it both ways. 


In [71]:
# Method 1
# your answer here
employee[["Department","Salary"]].groupby("Department").mean()


,Salary
Department,
HR,45.00
IT,48.75
Sales,55.00


In [72]:
# Method 2
# your answer here
#employee.groupby(['Salary']).mean()
employee.pivot_table(index=["Department"],values=['Salary'])

,Salary
Department,
HR,45.00
IT,48.75
Sales,55.00


#### We want now to group the data by more than one column. Use Pandas pivot table function to select the mean salary by department and title.

In [73]:
# your answer here
employee.pivot_table(index=["Department","Title"],values=["Salary"],aggfunc=["mean"]).head(5)

mean
                     Salary
Department Title           
HR         VP          70.0
           analyst     32.5
IT         VP          70.0
           analyst     32.5
           associate   60.0

#### We want to display the number of years of work experience and the salary for each Title. Add Years as a column in your pivot table.
*Hint: Use Pandas doc(see References in README.md) to explore the pivot_table() arguments*

In [74]:
# your answer here
employee.pivot_table(index=["Years","Title"], values=['Salary'])

Salary
Years Title            
1     analyst        35
2     analyst        30
      associate      55
3     associate      55
4     analyst        35
7     associate      60
8     VP             70

#### We want to know the information about the mean salary and number of employees in each department and for each job title.

In [75]:
# your answer here
employee.pivot_table(index=("Department","Title"), values=['Salary'], aggfunc=[np.mean,'count'])


mean  count
                     Salary Salary
Department Title                  
HR         VP          70.0      1
           analyst     32.5      2
IT         VP          70.0      1
           analyst     32.5      2
           associate   60.0      1
Sales      associate   55.0      2

#### We want to know for each department and for each job title the Salary's median, minimum, maximum and standard deviation. Comment your results.

In [76]:
# your answer here
employee.pivot_table(index=["Department","Title"], values=['Salary'], aggfunc=['median','min','max','std'])




median    min    max       std
                     Salary Salary Salary    Salary
Department Title                                   
HR         VP          70.0     70     70       NaN
           analyst     32.5     30     35  3.535534
IT         VP          70.0     70     70       NaN
           analyst     32.5     30     35  3.535534
           associate   60.0     60     60       NaN
Sales      associate   55.0     55     55  0.000000

#### Based on your comments, fill the missing values with an appropriate value.

In [77]:
# your answer here
employee.pivot_table(index=["Department"], columns=["Salary"], values=["Years"], fill_value=0)


Years              
Salary        30 35   55 60 70
Department                    
HR             2  4  0.0  0  8
IT             2  1  0.0  7  8
Sales          0  0  2.5  0  0

#### The stake holders want to know for each department the number of employees and how much money is spend on salaries. Could you provide that information? 

In [78]:
# your answer here
employee.pivot_table(values=['Salary'], columns=['Department'], aggfunc=['count'])

count         
Department    HR IT Sales
Salary         3  4     2

#### For each Department and Title, we want to know the the total years of work experience and the mean salary.

In [79]:
# your answer here
employee.pivot_table(index=["Years","Title"], values=['Salary'], aggfunc='mean')

Salary
Years Title            
1     analyst        35
2     analyst        30
      associate      55
3     associate      55
4     analyst        35
7     associate      60
8     VP             70

### Bonus

#### We now want to compute the mean salary after removing the maxima for each Department. Create the appropriate pivot table.

Hint: Write a custom function.

In [80]:
# your answer here


# Challenge 2

#### Open the ` Fitbit.csv` file and name your dataset `fitbit`.

In [81]:
# your answer here
fitbit=pd.read_csv(r"data\Fitbit.csv")
fitbit.head(5)


,Date,Calorie burned,Steps,Distance,Floors,Minutes Sedentary,Minutes Lightly Active,Minutes Fairly Active,Minutes Very Active,Activity Calories,MinutesOfSleep,MinutesOfBeingAwake,NumberOfAwakings,LengthOfRestInMinutes
0,08-05-2015,1934,905,0.65,0,1.355,46,0,0,1680,384,26,23,417
1,09-05-2015,3631,18925,14.11,4,611.000,316,61,60,2248,454,35,21,491
2,10-05-2015,3204,14228,10.57,1,602.000,226,14,77,1719,387,46,25,436
3,11-05-2015,2673,6756,5.02,8,749.000,190,23,4,9620,311,31,21,350
4,12-05-2015,2495,502,3.73,1,876.000,171,0,0,7360,407,65,44,491


#### Explore you dataset in terms of data types and descriptive statistics.
Hint: Use Pandas functions from previous lectures.

In [82]:
# your answer here
fitbit.dtypes


Date                       object
Calorie burned              int64
Steps                       int64
Distance                  float64
Floors                      int64
Minutes Sedentary         float64
Minutes Lightly Active      int64
Minutes Fairly Active       int64
Minutes Very Active         int64
Activity Calories           int64
MinutesOfSleep              int64
MinutesOfBeingAwake         int64
NumberOfAwakings            int64
LengthOfRestInMinutes       int64
dtype: object

In [83]:
# your answer here
fitbit.describe()

,Calorie burned,Steps,Distance,Floors,Minutes Sedentary,Minutes Lightly Active,Minutes Fairly Active,Minutes Very Active,Activity Calories,MinutesOfSleep,MinutesOfBeingAwake,NumberOfAwakings,LengthOfRestInMinutes
count,367.000000,367.000000,367.000000,367.000000,367.000000,367.000000,367.000000,367.000000,367.000000,367.000000,367.000000,367.000000,367.000000
mean,2741.501362,10121.588556,8.549128,11.724796,563.934482,236.405995,26.163488,35.722071,2044.147139,290.479564,28.008174,16.196185,321.343324
std,916.307036,5594.836225,3.409881,10.337370,294.793145,86.531376,20.319456,31.006682,2041.267168,154.752328,18.541415,10.757622,170.786726
min,179.000000,0.000000,0.000000,0.000000,1.002000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2698.000000,6730.500000,6.155000,5.000000,520.000000,179.000000,8.000000,10.500000,1218.500000,224.000000,14.000000,7.000000,248.000000
50%,2974.000000,10413.000000,8.290000,11.000000,663.000000,226.000000,24.000000,29.000000,1553.000000,337.000000,29.000000,16.000000,370.000000
75%,3233.000000,13916.500000,10.560000,16.000000,756.500000,290.000000,41.500000,54.000000,1927.500000,400.500000,41.500000,24.000000,440.500000
max,4351.000000,26444.000000,20.450000,101.000000,998.000000,472.000000,101.000000,153.000000,9830.000000,553.000000,78.000000,45.000000,607.000000


#### You suspect that there must be a linear relationship between the Minutes Very Active and the Steps. Compute the correlation between these variables.

In [84]:
# your answer here

fitbit[["Minutes Very Active","Steps"]].corr()




,Minutes Very Active,Steps
Minutes Very Active,1.000000,0.571452
Steps,0.571452,1.000000


#### Use matplotlib to visually plot Minutes Very active vs Steps. 

##### Hint : import matplotlib.pyplot as plt (See previous lessons)#####
- Use plt.scatter(x=df['col_name_1'], y=df['col_name_2'])

In [85]:
# your answer here
x=fitbit[["Minutes Very Active"]]
y=fitbit["Steps"]
plt.scatter(x,y)
plt.show()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\DELL\anaconda3\envs\Ironhack\lib\site-packages\IPython\core\interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\DELL\AppData\Local\Temp\ipykernel_816\676670808.py", line 4, in <module>
    plt.scatter(x,y)
  File "c:\Users\DELL\anaconda3\envs\Ironhack\lib\site-packages\matplotlib\pyplot.py", line 2780, in scatter
    noverlap=noverlap, pad_to=pad_to, sides=sides,
  File "c:\Users\DELL\anaconda3\envs\Ironhack\lib\site-packages\matplotlib\pyplot.py", line 2227, in gca
    and cbook._get_running_interactive_framework()):
  File "c:\Users\DELL\anaconda3\envs\Ironhack\lib\site-packages\matplotlib\pyplot.py", line 832, in gcf
  File "c:\Users\DELL\anaconda3\envs\Ironhack\lib\site-packages\matplotlib\_api\deprecation.py", line 454, in wrapper
    "parameter will become keyword-only %(removal)s.",
  File "c:\Users\DELL\anaconda3\envs\Ironhack\lib\site-packages\matplotlib\pyplot.p

#### What can you say about Minute Very Active and Steps? Write a comment below

In [86]:
# your answer here
##We can see that they are correlated but not so close,in my opinion.


#### We also suspect that there must be a linear relationship between the Minutes Sedentary and the Steps. Compute the correlation between these variables.

In [87]:
# your answer here
fitbit[["Minutes Sedentary","Steps"]].corr()


,Minutes Sedentary,Steps
Minutes Sedentary,1.000000,0.076906
Steps,0.076906,1.000000


#### Use matplotlib to visually plot Minutes Sedentary vs Steps. Based on the results of the computed correlation and the plot. What can you say about these 2 variables?

In [88]:
# your answer here
x=fitbit[["Minutes Sedentary"]]
y=fitbit["Steps"]
plt.scatter(x,y)
plt.show()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\DELL\anaconda3\envs\Ironhack\lib\site-packages\IPython\core\interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\DELL\AppData\Local\Temp\ipykernel_816\3650516092.py", line 4, in <module>
    plt.scatter(x,y)
  File "c:\Users\DELL\anaconda3\envs\Ironhack\lib\site-packages\matplotlib\pyplot.py", line 2780, in scatter
    noverlap=noverlap, pad_to=pad_to, sides=sides,
  File "c:\Users\DELL\anaconda3\envs\Ironhack\lib\site-packages\matplotlib\pyplot.py", line 2227, in gca
    and cbook._get_running_interactive_framework()):
  File "c:\Users\DELL\anaconda3\envs\Ironhack\lib\site-packages\matplotlib\pyplot.py", line 832, in gcf
  File "c:\Users\DELL\anaconda3\envs\Ironhack\lib\site-packages\matplotlib\_api\deprecation.py", line 454, in wrapper
    "parameter will become keyword-only %(removal)s.",
  File "c:\Users\DELL\anaconda3\envs\Ironhack\lib\site-packages\matplotlib\pyplot.

#### We also suspect that there must be a linear relationship between the MinutesOfSleep and the Steps. Compute the correlation between these variables.

In [89]:
# your answer here

fitbit[["MinutesOfSleep","Steps"]].corr()





,MinutesOfSleep,Steps
MinutesOfSleep,1.000000,0.130986
Steps,0.130986,1.000000


#### Use matplotlib to visually plot MinutesOfSleep vs Steps. Based on the results of the computed correlation and the plot. What can you say about these 2 variables?

In [90]:
# your answer here
x=fitbit[["MinutesOfSleep"]]
y=fitbit["Steps"]
plt.scatter(x,y)
plt.show()


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\DELL\anaconda3\envs\Ironhack\lib\site-packages\IPython\core\interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\DELL\AppData\Local\Temp\ipykernel_816\3085085665.py", line 4, in <module>
    plt.scatter(x,y)
  File "c:\Users\DELL\anaconda3\envs\Ironhack\lib\site-packages\matplotlib\pyplot.py", line 2780, in scatter
    noverlap=noverlap, pad_to=pad_to, sides=sides,
  File "c:\Users\DELL\anaconda3\envs\Ironhack\lib\site-packages\matplotlib\pyplot.py", line 2227, in gca
    and cbook._get_running_interactive_framework()):
  File "c:\Users\DELL\anaconda3\envs\Ironhack\lib\site-packages\matplotlib\pyplot.py", line 832, in gcf
  File "c:\Users\DELL\anaconda3\envs\Ironhack\lib\site-packages\matplotlib\_api\deprecation.py", line 454, in wrapper
    "parameter will become keyword-only %(removal)s.",
  File "c:\Users\DELL\anaconda3\envs\Ironhack\lib\site-packages\matplotlib\pyplot.

#### Select a subset of your dataset with the columns below and compute the correlation matrix. Save the results in the variable `cor_fit`
column= ['Calorie burned', 'Steps','Floors','Minutes Sedentary','Minutes Very Active', 'MinutesOfSleep']

In [91]:
# your answer here
cor_fit=fitbit[["Calorie burned","Steps","Floors","Minutes Sedentary","Minutes Very Active","MinutesOfSleep"]]
cor_fit.head(5)

,Calorie burned,Steps,Floors,Minutes Sedentary,Minutes Very Active,MinutesOfSleep
0,1934,905,0,1.355,0,384
1,3631,18925,4,611.000,60,454
2,3204,14228,1,602.000,77,387
3,2673,6756,8,749.000,4,311
4,2495,502,1,876.000,0,407


#### Based on the result of the correlation matrix. What are the highly correlated features with the variable Steps?
*Hint: From the results above select the variable Steps and order your results in descending order (use Pandas sort_values function)*

In [92]:
# your answer here
cor_fit.sort_values(["Steps"],ascending=True)

,Calorie burned,Steps,Floors,Minutes Sedentary,Minutes Very Active,MinutesOfSleep
326,179,0,0,1.440,0,0
311,179,0,0,1.440,0,0
229,1789,0,0,1.440,0,0
81,298,10,13,803.000,11,472
261,1801,39,0,1.076,0,0
...,...,...,...,...,...,...
335,4112,23286,19,561.000,67,267
206,4012,23313,16,550.000,75,69
314,396,25385,17,638.000,124,368
357,403,25571,15,606.000,129,374


# Challenge 3

#### Open the `Time_Grades.csv` file and name your dataset `time_grades`.

In [93]:
# your answer here
time_grades=pd.read_csv(r"data\Time_Grades.csv")
time_grades.head(5)

,Name,Study time,Grade
0,Jose,4,4
1,Maria,9,7
2,David,8,9
3,Sonia,10,7
4,Samuel,20,9


#### Show visually the frequency distribution of the time_grades dataset.

In [99]:
# your answer here
time_grades.hist()

ImportError: cannot import name '_check_savefig_extra_args' from 'matplotlib.backend_bases' (c:\Users\DELL\anaconda3\envs\Ironhack\lib\site-packages\matplotlib\backend_bases.py)

#### Is there a correlation between study time and Grade? Use both Pearson and Spearman correlation and comment your results.

In [100]:
# your answer here
time_grades[["Study time","Grade"]].corr()


,Study time,Grade
Study time,1.000000,0.798046
Grade,0.798046,1.000000


#### Use matplotlib to visually plot Study time vs Grade. Based on the results of the computed correlation and the plot. What can you say about these 2 variables?

In [101]:
# your answer here
x=time_grades[["Study time"]]
y=["Grade"]
plt.scatter(x,y)
plt.show()

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\DELL\anaconda3\envs\Ironhack\lib\site-packages\IPython\core\interactiveshell.py", line 3378, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\DELL\AppData\Local\Temp\ipykernel_816\2368307925.py", line 4, in <module>
    plt.scatter(x,y)
  File "c:\Users\DELL\anaconda3\envs\Ironhack\lib\site-packages\matplotlib\pyplot.py", line 2780, in scatter
    noverlap=noverlap, pad_to=pad_to, sides=sides,
  File "c:\Users\DELL\anaconda3\envs\Ironhack\lib\site-packages\matplotlib\pyplot.py", line 2227, in gca
    and cbook._get_running_interactive_framework()):
  File "c:\Users\DELL\anaconda3\envs\Ironhack\lib\site-packages\matplotlib\pyplot.py", line 832, in gcf
  File "c:\Users\DELL\anaconda3\envs\Ironhack\lib\site-packages\matplotlib\_api\deprecation.py", line 454, in wrapper
    "parameter will become keyword-only %(removal)s.",
  File "c:\Users\DELL\anaconda3\envs\Ironhack\lib\site-packages\matplotlib\pyplot.

#### Could you explain the difference between Pearson and Spearman correlation? (see References in README.md) 

In [97]:
# your answer here
